In [1]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split # Import train_test_split function
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [3]:
df = pd.read_csv('DeathGripsDataset.csv')
df.head()

,Unnamed: 0,names,albums,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo,types,duration_ms,time_signature,Labels
0,0,Get Got,The Money Store,0.570,0.993,7,-3.757,1,0.337,0.00628,0.007960,0.405,87.000,audio_features,171867,4,1
1,1,The Fever (Aye Aye),The Money Store,0.432,0.940,10,-4.099,0,0.374,0.15900,0.000000,0.225,80.084,audio_features,186973,4,1
2,2,Lost Boys,The Money Store,0.464,0.949,2,-2.454,1,0.336,0.00300,0.000113,0.374,77.121,audio_features,186320,4,1
3,3,Blackjack,The Money Store,0.532,0.855,4,-3.014,0,0.301,0.25900,0.000000,0.534,136.902,audio_features,142000,4,1
4,4,Hustle Bones,The Money Store,0.600,0.982,4,-2.996,0,0.201,0.10400,0.000000,0.277,110.015,audio_features,192360,4,1


In [4]:
yescols = ["danceability", "key",
       "loudness", "mode", "speechiness", "acousticness", "instrumentalness",
       "valence", "tempo"]
nocols = ['names','albums', "energy", 'types', 'duration_ms', 'time_signature']

In [5]:
X = np.array(df[yescols])
Y = np.array(df.Labels)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42) # 70% training and 30% test

In [6]:
class Data(Dataset):
  def __init__(self, X_train, y_train):
    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
    self.X = torch.from_numpy(X_train.astype(np.float32))
    # need to convert float64 to Long else 
    # will get the following error
    # RuntimeError: expected scalar type Long but found Float
    self.y = torch.from_numpy(y_train).type(torch.LongTensor)
    self.len = self.X.shape[0]
  
  def __getitem__(self, index):
    return self.X[index], self.y[index]
  def __len__(self):
    return self.len

In [7]:
traindata = Data(X_train, Y_train)
print(traindata[0])

(tensor([ 4.4800e-01,  1.0000e+00, -2.6200e+00,  1.0000e+00,  3.9400e-01,
         7.3100e-02,  0.0000e+00,  4.9600e-01,  1.5384e+02]), tensor(0))


In [8]:
batch_size = 1
trainloader = DataLoader(traindata, batch_size=batch_size, 
                         shuffle=True, num_workers=0)

In [9]:
# number of features (len of X cols)
input_dim = len(yescols)
# number of hidden layers
hidden_layers = 4
# number of classes (unique of y)
output_dim = 2
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.linear1 = nn.Linear(input_dim, hidden_layers)
    self.linear2 = nn.Linear(hidden_layers, output_dim)
  def forward(self, x):
    x = torch.sigmoid(self.linear1(x))
    x = self.linear2(x)
    return x

In [10]:
clf = Network()

In [11]:
print(clf.parameters)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(clf.parameters(), lr=0.1)

<bound method Module.parameters of Network(
  (linear1): Linear(in_features=9, out_features=4, bias=True)
  (linear2): Linear(in_features=4, out_features=2, bias=True)
)>


In [12]:
epochs = 100
for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    # set optimizer to zero grad to remove previous epoch gradients
    optimizer.zero_grad()
    # forward propagation
    outputs = clf(inputs)
    loss = criterion(outputs, labels)
    # backward propagation
    loss.backward()
    # optimize
    optimizer.step()
    running_loss += loss.item()
  # display statistics
  print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.5f}')

[1,    61] loss: 0.02127
[2,    61] loss: 0.02199
[3,    61] loss: 0.02173
[4,    61] loss: 0.02171
[5,    61] loss: 0.02182
[6,    61] loss: 0.02195
[7,    61] loss: 0.02196
[8,    61] loss: 0.02116
[9,    61] loss: 0.02123
[10,    61] loss: 0.02142
[11,    61] loss: 0.02175
[12,    61] loss: 0.02190
[13,    61] loss: 0.02145
[14,    61] loss: 0.02139
[15,    61] loss: 0.02161
[16,    61] loss: 0.02161
[17,    61] loss: 0.02182
[18,    61] loss: 0.02153
[19,    61] loss: 0.02187
[20,    61] loss: 0.02152
[21,    61] loss: 0.02175
[22,    61] loss: 0.02148
[23,    61] loss: 0.02192
[24,    61] loss: 0.02193
[25,    61] loss: 0.02174
[26,    61] loss: 0.02129
[27,    61] loss: 0.02146
[28,    61] loss: 0.02079
[29,    61] loss: 0.02160
[30,    61] loss: 0.02159
[31,    61] loss: 0.02167
[32,    61] loss: 0.02154
[33,    61] loss: 0.02186
[34,    61] loss: 0.02183
[35,    61] loss: 0.02169
[36,    61] loss: 0.02175
[37,    61] loss: 0.02118
[38,    61] loss: 0.02197
[39,    61] loss: 0.0

In [13]:
# save the trained model
PATH = './mymodel.pth'
torch.save(clf.state_dict(), PATH)

In [14]:
clf = Network()
clf.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [15]:
testdata = Data(X_test, Y_test)
testloader = DataLoader(testdata, batch_size=batch_size, 
                        shuffle=True, num_workers=0)

In [16]:
dataiter = iter(testloader)
inputs, labels = next(dataiter)

In [17]:
print(inputs)
print(labels)

tensor([[ 6.4100e-01,  6.0000e+00, -4.4870e+00,  0.0000e+00,  1.2400e-01,
          8.5400e-02,  0.0000e+00,  3.1300e-01,  1.0704e+02]])
tensor([1])


In [18]:
outputs = clf(inputs)
__, predicted = torch.max(outputs, 1)
print(predicted)

tensor([0])


In [19]:
correct, total = 0, 0
# no need to calculate gradients during inference
with torch.no_grad():
  for data in testloader:
    inputs, labels = data
    # calculate output by running through the network
    outputs = clf(inputs)
    # get the predictions
    __, predicted = torch.max(outputs.data, 1)
    # update results
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the {len(testdata)} test data: {100 * correct // total} %')

Accuracy of the network on the 27 test data: 55 %
